<a href="https://colab.research.google.com/github/LamuneGitHub/A001_Python_Test/blob/main/EDA_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----
----
# # EDA ( Exploratory Data Analysis )( 탐색적 데이터 분석)
## 정의
> : 탐색적 데이터 분석  
: 데이터의 특징을 파악하는것

## 목적
> : 본격적인 데이터 분석이나 모델링 전에 데이터의 특징을 파악하여 분석 계획을 세우기 위한 기초공사 작업  
  : 유의미한 business insight(업무 통찰)을 얻기 위함  
  : 시행착오 비용 최소화

## 수행 과정 및 방법
* Data Description 확인 (데이터 명세서 )  
  : 데이터 명세서를 읽어서 데이터의 특성을 이해 

* 시각적 분석
  * 엑셀등을 이용하여 직접 눈으로 특성을 분석

* 프로그램적 분석
  * 분석 메소드 활용
    * info()
    * shape
    * description()
    * isnull().sum()
    * duplicated()
    * 기타 프로그램코딩 활용
  * Feature Engineering 
  * 시각화

> 목표 (프로그램적 분석의)
  * 결측치 확인
    * 컬럼별 결측치 비율
  * 카테고리 값의 비율
  * 가능 범위를 벗어난 값 확인
  * 계산값이 틀린 값 확인
  * 이상한 문자가 들어간 값 확인
  * 중복값

* Feature Engineering (변수 가공)
* Business Insight (업무 통찰)



# # 프로그램 기본 설정

In [5]:
# import

%matplotlib inline

from IPython.display import display

import re
import math
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#pandas에서 DataFrame을 요약해서 표시하지 않도록 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# head로 표시할 데이터 갯수 
print_row_count = 3

In [6]:
# # 구글 드라이브 mount
# from google.colab import drive
# drive.mount('/content/drive');


# # 함수 정의

In [ ]:
# 줄 구분 표시 출력 
def print_line() :
  print ("\n------------------------------------------------")
def print_line_m() :
  print ("\n-----------------------")
def print_line_s() :
  print ("\n----------")

In [7]:
# 결측치가 포함된 df를 반환 한다. ( copy 된 data)
"""
    str_replace = np.nan # 결측치를 변경할 문자
    ,need_replace = False # 결측치 변경이 필요한지 여부 
    ,need_drop = False # 결측치 행의 삭제가 필요한지 여부
"""
def 결측치행열조회( df_target , str_replace = np.nan , need_replace = False , need_drop = False) :

    # 결측치로 인식할 문자열
    #str_replace = '-' # 결측치를 변경할 문자
    lst_na_str = [ np.NaN , '-' , "N/A" 'NA' , 'na', 'NaN' , "" ]   # 결측치로 인식할 문자들 

    # 조회
    cond_tmp1 = df_target.isin(lst_na_str).any( axis= 1)

    # isnull()의 결과는 위에 포함 되므로 반복 처리 해 주지 않아도 된다. 
    """
    # 빈값 조회
    cond_tmp2 = df_target.isnull().any(axis= 1)

    # 빈값 이거나, 빈값 문자열인 경우
    cond_tmp3 = cond_tmp1 | cond_tmp2

    set_1 = set(trueIndex리스트반환( cond_tmp1 ))
    set_2 = set(trueIndex리스트반환( cond_tmp2 ))

    print ( set_1 > set_2 )
    print ( set_1 == set_2 )
    print ( set_1 < set_2 )"""

    # 결측치가 포함된 컬럼명 조회
    sri_tmp = df_target.isin(lst_na_str).any()
    list_col = sri_tmp[sri_tmp == True].index

    # 작업 Df 생성
    df_tmp = df_target[cond_tmp1][list_col].copy()
    #display ( df_tmp  )
    #display ( df_target[cond_tmp1][list_col]  )

    # 출력
    print ( f"결측치가 포함된 Data 갯수 {len(df_tmp)} / { len(df_target)} " )
    print ( f"각 컬럼별 결측치 갯수")
    print ( f"{df_tmp.isin(lst_na_str).sum()}")

    # (TODO)결측치로 인식한 문자열을  str_replace 으로 변경
    if need_replace :
        df_tmp.replace( np.nan , str_replace , inplace=True)

    # 결측치 행 삭제
    if need_drop :
        df_target.drop(df_tmp.index , inplace=True)

    return df_tmp

In [8]:
# EDA 기본정보 자동 분석 함수
def eda_info (df_param, need_category = False , category_count = 20) :

  df_target = df_param
  
  # df 변형이 필요한 경우 copy 
  if need_category :
    df_target = df_param.copy()
  
  list_columns = df_target.columns

  # 데이터셋 정보 확인
  #: 컬럼 자료형 , 전체 자료 갯수 , 컬럼 갯수 , 결측치 여부 , 전체 data 크기
  print_line()
  print ( "# 데이터셋 정보 확인" )
  print ( df_target.info() )


  # 컬럼 자료형 조회 (dtypes 이용) 
  #df_titanic.dtypes


  # 데이터의 행열 크기 조회
  print_line()
  print( "# 데이터의 행열 크기 = " , df_target.shape )


  # 결측치가 존재하는 컬럼명 조회
  print_line()
  """
  print( "# 컬럼별 결측치 갯수 ")
  print( df_target.isnull().sum() )

  print_line()
  print( "# 결측치 총 갯수 = " , df_target.isnull().sum().sum())
  """
  df_null = 결측치행열조회( df_target )
  print ( df_null.head(print_row_count) )


  # 컬럼별 중복값 확인
  print_line()
  print( "# 컬럼별 중복값 확인 ")
  for tmp_column in list_columns :
    # 중복값 갯수 조회
    tmp_dup_count = df_target[tmp_column].duplicated(keep = False).sum()
    
    if tmp_dup_count > 0 :
      print ( tmp_column , "컬럼에 중복값", tmp_dup_count , "개" )
      #print ( "index = " , df_target[tmp_column].duplicated(keep = False)   )
      #TODO : 중복값 인덱스 출력
    
  # 중복 데이터 확인
  print( "중복 data 갯수(전체)" , df_target.duplicated(keep = False).sum() )
  print( "중복 data " , df_target[df_target.duplicated(keep = False)].head(print_row_count) )


  # 각 컬럼별 값의 비율 조회
  # 컬럼명 조회
  print_line()
  print( "# 컬럼별 값의 비율 ")
  print ("컬럼 목록= " , list_columns)

  # 컬럼별 값의 비율 조회
  list_category_col_name = [] # 카테고리화 가능할 것으로 예상되는 컬럼명
  for tmp_column in list_columns :
    df_temp = df_target[tmp_column].value_counts(normalize=True ,dropna = True)
    
    print_line_s()
    print ( f"컬럼명 = {tmp_column} , 값의종류 갯수 = {len(df_temp)} " )
    
    if len(df_temp) <= category_count :
      list_category_col_name.append( tmp_column )
      print ( "Category 가능성 있음")
      print ( df_temp )
    #else :
      #print ( "값의 종류가 20개 이상임" )


  # 카테고리화 가능한 컬럼은 카테고리화 진행 
  if need_category :
    print ( "카테고리화 진행")
    for col_name in list_category_col_name :
      df_target[col_name] = df_target[col_name].astype('category')


  # 컬럼별 통계치 확인
  print_line()
  print ( "#컬럼별 통계특성 확인" )
  print ( df_target.describe())


  # 컬럼별 포함된 큭수문자 의 종류 조회
  print_line()
  print( "# 컬럼별 포함된 특수문자 종류 ")
  regx_특수문자제외한 = r'([^0-9a-zA-Z]+)'
  for tmp_column in list_columns :

    if df_target[tmp_column].dtype == 'object'  :
      print ( "컬럼명 = " , tmp_column )
      
      df_rslt = df_target[tmp_column].str.extractall( r'([^0-9a-zA-Z]+)'  )
      #display ( type(df_rslt) ) #=> DataFrame 

      set_rslt = []
      for idx in df_target.index :
        
        # TODO 값이 Nan인 경우의 정규표현식을 조사하지 않도록 한다,
        # 우선은 str인 경우만 
        tmp_val = df_target.loc[ idx , tmp_column]
        if isinstance( tmp_val , str) : 
          rslt = re.findall ( regx_특수문자제외한 , df_target.loc[ idx , tmp_column] )
          set_rslt += rslt 
        #else :
        #  display ( tmp_val )

      print( set(set_rslt) )
      
      print_line_s()


  # category 컬럼의 각 값 별 갯수와 비율 조회
  print_line()
  print ( "#category 타입 컬럼의 각 값 갯수와 비율 조회" )
  list_category_col_name = df_target.dtypes[df_target.dtypes.values == 'category'].index.tolist() 
  for col_name in list_category_col_name :
    sri_a = df_target[col_name].value_counts(dropna = True)
    sri_b = df_target[col_name].value_counts(normalize=True ,dropna = True)
    df_a = sri_a.to_frame( name = 'count')
    df_b = sri_b.to_frame( name = 'ratio')

    print (f"컬럼명 = {col_name} ")
    print ( df_a.join( df_b ) )
    print_line_s()

  #TODO 컬럼별 값 분포 그래프 표시
  #Boxplot과 Histogram을 통해서도 이상치 존재 여부를 확인해볼 수 있습니다.

  return df_target

In [115]:
# 숫자와 . 이외에 어떤 문자가 들어가 있는지 확인 하고 문제가 있는 Data를 df 형식으로 반환
# 
def 숫자이외의문자확인 (df_target , list_columns ) :
  list_err_idx = []
  regx_숫자를제외한 = r'([^0-9.]+)'
  for tmp_column in list_columns :

    if df_target[tmp_column].dtype == 'object'  :
      print ( "컬럼명 = " , tmp_column )
      
      df_rslt = df_target[tmp_column].str.extractall( regx_숫자를제외한  )
      #display ( type(df_rslt) ) #=> DataFrame 

      list_err_str = [] 
      for idx in df_target.index :
        tmp_val = df_target.loc[ idx , tmp_column]

        # TODO 값이 Nan인 경우의 정규표현식을 조사하지 않도록 하는 조건 추가
        # 우선은 str인 경우만 체크 진행 하도록 하였음
        if isinstance( tmp_val , str) : 

          rslt = re.findall ( regx_숫자를제외한 , df_target.loc[ idx , tmp_column] )
          if len(rslt) != 0 :
            list_err_str += rslt
            list_err_idx.append( idx )

        #else :
        #  display ( tmp_val )

      print( set(list_err_str) )
      
      print_line_s()

  err_idx = sorted( set(list_err_idx)) 

  # err 데이터 반환
  return df_target.loc[err_idx]

In [10]:
# DataFrame의 특정 컬럼의 data중에서 cond_sel 조건에 맞는 data를 fix_func 으로 수정 후 수정된 DataFrame을 반환
def fix_column_err_value ( df_temp_clean: pd.DataFrame , target_column: '수정할 column 명' 
                          , cond_sel:'수정 데이터 check 조건' 
                          , fix_func:'err를 수정할 함수' 
                          ) -> pd.DataFrame : #'수정된 DataFrame' 

    # err 조건에 해당하는 data 조회
    sri_err = df_temp_clean[ target_column ].apply( cond_sel )

    # 조회 결과 확인 
    display ( "수정전")
    display ( sri_err.sum() )
    display ( df_temp_clean.loc[ sri_err ].head(print_row_count) )


    # 오류를 수정 후 data 확인
    #(TODO) sri_fixed_col = df_temp_clean.loc[ sri_err ][[target_column]].apply( fix_func ,axis=1)
    sri_fixed_col = df_temp_clean.loc[ sri_err ][target_column].apply( fix_func )
    #display (sri_fixed_col.head(3))

    # 오류 수정한 데이터를 전처리 작업용 DataFrame에 반영
    df_temp_clean.loc[ sri_err, target_column ] = sri_fixed_col

    # 반영 결과 data 확인 
    display ( "반영결과")
    display ( df_temp_clean[sri_err].head(print_row_count))

    # # 반영 결과 집계 확인 
    # sri_err = df_temp_clean[ target_column ].apply( cond_sel )
    # display ( sri_err.sum() )

    return df_temp_clean

## 작업 DataFrame 지정

In [11]:
# 데이터 로딩

# # 엑셀 파일 로딩
# df_001 = pd.read_excel("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/stocks/Travel.xlsx", sheet_name = "008770 ")
# # CSV 파일 로딩
# df_red = pd.read_csv( "https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv" , sep = ",")
# df_white = pd.read_csv( "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv" , sep = ";")

# df_titanic = sns.load_dataset("titanic")

# df_patients = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/patients_info.csv')
# df_insuline_test = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/insuline_test.csv')

# df_final_revised = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/final_revised.csv')
# df_movie = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/movie.csv')

# df_discussion_114 = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/Discussion_114.csv')



# df_data = pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2020.03.csv', encoding='euc-kr', usecols=['측정일시', '측정소명', '오존(ppm)'])

# read_csv 파라미터
"""    ,sep=','
    ,encoding='cp949' , encoding='euc-kr' , encoding='ISO-8859-1'
    , usecols=['측정일시', '측정소명', '오존(ppm)']
    , header=0 
    , index_col=0
"""

"    ,sep=','\n    ,encoding='cp949' , encoding='euc-kr' , encoding='ISO-8859-1'\n    , usecols=['측정일시', '측정소명', '오존(ppm)']\n    , header=0 \n    , index_col=0\n"

----
----
# # EDA 수행

## 1. 시각적 EDA 수행

In [95]:
df_tmp = pd.read_csv( "https://raw.githubusercontent.com/LamuneGitHub/AI15_Prj_01/main/vgames2.csv" , sep=","  , header=0  , index_col=0 ) 
df_target = df_tmp

#맨위 
display( df_target.head(print_row_count) )
#마지막
display( df_target.tail(print_row_count) )
#무작위
display( df_target.sample(print_row_count) )


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03
16598,The King of Fighters '95,PS,1996.0,Fighting,Sony Computer Entertainment,0,0,0.16,0.01


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
717,Whiteout,XB,2002.0,Racing,Konami Digital Entertainment,0.02,0.01,0,0
16231,Battle Arena Toshinden URA,SAT,1995.0,Fighting,Sega,0,0,0.03,0
2490,Kamen Rider: Travelers Senki,3DS,2013.0,Action,Seventh Chord,0,0,0.08,0


----


## 2. 프로그램적 EDA 진행 기록

## 2.1 프로그램을 사용하여 데이터 특성 분석

In [96]:
# 데이터 자동확인 리포트 함수 수행 
#
# 자동 카테고리화 진행
df_target = eda_info (df_target, need_category= True , category_count = 50) 
print( df_target.info() ) # 자동 카테고리화 이후의 결과 확인
display( df_target.head(print_row_count) ) # 자동 카테고리화 이후의 결과 확인


------------------------------------------------
# 데이터셋 정보 확인
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB
None

------------------------------------------------
# 데이터의 행열 크기 =  (16598, 9)

------------------------------------------------
결측치가 포함된 Data 갯수 357 / 16598 
각 컬럼별 결측치 갯수
Year         271
Genre         50
Publisher     58
dtype: int64
       Year   Genre                    Publisher
32      NaN    Misc        

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0


## 2.2 데이터 정제
  
    반복 {
        2.2.1 문제점 리스팅
        2.2.2 문제점 해결
               & 결과 확인 
    }

In [97]:
# 도출된 문제점 수정


#(v) 결측치 확인, 처리  -> 삭제 , 357 / 16598
df_null = 결측치행열조회(df_target, need_drop = True   )

#(v) Name 중복 컬럼 확인 -> OK
"""
df_target[df_target['Name'].duplicated(keep = False)]
df_target[df_target['Name'] == "Candace Kane's Candy Factory"]
"""

#(v) 중복값 삭제 -> 1개 삭제
df_target.drop_duplicates(inplace=True)

# Year 수정 별도로 좀 더 자세히 하는것으로 변경 하였음 ( 아래 확인 )
"""
#(v) Year 값이 0 인 이상치 수정. -> 3개 , 매출값도 크지 않음 -> 삭제
cond_tmp = df_target['Year'] == 0
len (df_target[cond_tmp])
df_target.drop(df_target[cond_tmp].index, inplace=True)
"""
#(v) Year -> int 로 수정
#(TODO) float 컬럼의 값이 모두 .0 형식(int) 형식인지 체크  
# int로 type변경
df_target['Year'] = df_target['Year'].astype( 'int')

#(v) Publisher 컬럼 필요 없으므로 Drop
#(v) Name 컬럼 필요 없으므로 Drop
df_target.drop( ['Publisher','Name' ] , axis='columns' , inplace=True)

# 데이터 재 확인
# df_target = eda_info (df_target) 


결측치가 포함된 Data 갯수 357 / 16598 
각 컬럼별 결측치 갯수
Year         271
Genre         50
Publisher     58
dtype: int64


In [98]:
# # df 백업
# df_backup_copy = df_target.copy()

In [99]:
# df 복원
#df_target = df_backup_copy.copy()

In [100]:
#[v] Sales 컬럼에 들어있는 특수 문자 , 영문자 확인 및 삭제 , int로 변환

# 이름에 Sales 가 포함된 컬럼을 대상으로 함 
list_sales_col_name = [ col for col in df_target.columns if col.find("Sales") != -1  ]  
print  (list_sales_col_name )
list_columns = list_sales_col_name

# 컬럼에 숫자 이외의 문자가 있는지 여부 확인
숫자이외의문자확인 (df_target , list_columns );


['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
컬럼명 =  NA_Sales
{'M', 'K'}

----------
컬럼명 =  EU_Sales
{'M', 'K'}

----------
컬럼명 =  JP_Sales
{'M', 'K'}

----------
컬럼명 =  Other_Sales
{'M', 'K'}

----------


In [101]:
# K가 들어가 있는 컬럼은 * 1000 
# M이 들어가 있는 컬럼은 * 10000000  = 10^6
# K와 Md이 없는 컬럼은 M과 똑같이 * 1000000 = 10^6
#   해준후 int로 컬럼 data type을 변경한다.

# err 데이터 확인
#print( df_target.loc[err_idx] )

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
cond_all = lambda x: True 
#cond_contain_M = lambda x: True if re.search( 'M' , x).group(0) else False 

# err 수정 function.
def fix_func( x ):
  fix_value = x
  try:

    if re.search( 'K' , x ) :
      fix_value = x.replace ( "K" , "" )
      fix_value = float(fix_value) * 10**3    
    elif re.search( 'M' , x ) :
      fix_value = x.replace ( "M" , "" )
      fix_value = float(fix_value) * 10**6
    else :
      fix_value = float(fix_value) * 10**6

    fix_value = int(fix_value)
    return fix_value
  except Exception:
    print( f"에러 발생 {x} , { type( x )} ")

# 수정 함수 호출
for target_column in list_columns :
  df_target = fix_column_err_value( df_target , target_column , cond_all, fix_func ) 

'수정전'

16241

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,0.04,0,0,0
2,Wii,2009,Action,0.17,0,0,0.01
3,PSP,2010,Adventure,0,0,0.02,0


'반영결과'

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,0.01
3,PSP,2010,Adventure,0,0,0.02,0


...

'수정전'

16241

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,0.01
3,PSP,2010,Adventure,0,0,20000,0


'반영결과'

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,10000
3,PSP,2010,Adventure,0,0,20000,0


In [102]:
a = 숫자이외의문자확인 (df_target , list_columns )

In [103]:
# 데이터 클리닝 결과 재확인
df_target = eda_info (df_target) 


------------------------------------------------
# 데이터셋 정보 확인
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Platform     16241 non-null  category
 1   Year         16241 non-null  int64   
 2   Genre        16241 non-null  category
 3   NA_Sales     16241 non-null  int64   
 4   EU_Sales     16241 non-null  int64   
 5   JP_Sales     16241 non-null  int64   
 6   Other_Sales  16241 non-null  int64   
dtypes: category(2), int64(5)
memory usage: 1.3 MB
None

------------------------------------------------
# 데이터의 행열 크기 =  (16241, 7)

------------------------------------------------
결측치가 포함된 Data 갯수 0 / 16241 
각 컬럼별 결측치 갯수
Series([], dtype: float64)
Empty DataFrame
Columns: []
Index: []

------------------------------------------------
# 컬럼별 중복값 확인 
Platform 컬럼에 중복값 16239 개
Year 컬럼에 중복값 16238 개
Genre 컬럼에 중복값 16241 개
NA_Sales 컬럼에 중복값 1608

In [104]:
#(v) Year 최저 값이 이상한 것을 발견 하였음 
print( sorted(df_target.Year.unique()) )

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 86, 94, 95, 96, 97, 98, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2020]


In [105]:
# Year 값 수정
"""Year 
    0~ 16  => 2000년도
    86~98 => 1900년도
"""

target_column = 'Year'

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
cond_all = lambda x: True 

# err 수정 function.
def fix_func( x ):
  fix_value = x
  try:

    if x <= 16  :
      fix_value = x + 2000    
    elif x <= 98  :
      fix_value = x + 1900
    
    fix_value = int(fix_value)
    return fix_value
  except Exception:
    print( f"에러 발생 {x} , { type( x )} ")

# 수정 함수 호출
df_target = fix_column_err_value( df_target , target_column , cond_all, fix_func ) 



'수정전'

16241

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,10000
3,PSP,2010,Adventure,0,0,20000,0


'반영결과'

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,10000
3,PSP,2010,Adventure,0,0,20000,0


In [106]:
# 데이터 클리닝 결과 재확인
df_target = eda_info (df_target) 


------------------------------------------------
# 데이터셋 정보 확인
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Platform     16241 non-null  category
 1   Year         16241 non-null  int64   
 2   Genre        16241 non-null  category
 3   NA_Sales     16241 non-null  int64   
 4   EU_Sales     16241 non-null  int64   
 5   JP_Sales     16241 non-null  int64   
 6   Other_Sales  16241 non-null  int64   
dtypes: category(2), int64(5)
memory usage: 1.3 MB
None

------------------------------------------------
# 데이터의 행열 크기 =  (16241, 7)

------------------------------------------------
결측치가 포함된 Data 갯수 0 / 16241 
각 컬럼별 결측치 갯수
Series([], dtype: float64)
Empty DataFrame
Columns: []
Index: []

------------------------------------------------
# 컬럼별 중복값 확인 
Platform 컬럼에 중복값 16239 개
Year 컬럼에 중복값 16240 개
Genre 컬럼에 중복값 16241 개
NA_Sales 컬럼에 중복값 1608

----
데이터 클리닝 완료

----

## 3. Feature Engineering
: 수학적인 연산과 도메인 지식을 이용하여 raw data로부터 유용한 feature를 도출해 내는 과정

In [107]:
# # df 백업
# df_backup_copy = df_target.copy()

In [108]:
# df_target = df_backup_copy.copy()

In [109]:
df_target.head(5)

,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,DS,2008,Action,40000,0,0,0
2,Wii,2009,Action,170000,0,0,10000
3,PSP,2010,Adventure,0,0,20000,0
4,DS,2010,Misc,40000,0,0,0
5,PS3,2010,Platform,120000,90000,0,40000


In [110]:
# 플렛폼을 모바일 , 콘솔 , 컴퓨터의 대분류로도 구분
import sys 

lst_portable = [ 'DS', '3DS', 'PSP', 'GB', 'GBA', 'PSV', 'WS', 'GG', 'WiiU']
lst_pc = ['PC']
list_all = df_target.Platform.unique().tolist()
lst_console = [ x for x in list_all if (x not in lst_portable) & (x not in lst_pc) ]
print ( lst_console) # ['Wii', 'PS3', 'PS', 'PS4', 'PS2', 'XB', 'X360', 'GC', '2600', 'SAT', 'NES', 'DC', 'N64', 'XOne', 'SNES', 'GEN', 'SCD', 'NG', 'TG16', '3DO', 'PCFX']


target_column = 'Platform'

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
cond_SCD = lambda x: True if x == 'SCD' else False 
cond_all = lambda x: True

# 수정 function.
def fix_func( x ):
  
  ret_value = "-"

  try:
    
    if x['Platform'] in lst_portable  :
      ret_value = 'portable'
    elif x['Platform'] in lst_console  :
      ret_value = 'console'
    elif x['Platform'] in lst_pc  :
      ret_value = 'pc'
    
    df_target.loc[x.name,"Type"] = ret_value
   
    return x.str
  except Exception:
    print( f"에러 발생 data = {x} , type = { type( x )} , idx = {x.name} ")
    sys.exit()



######

# err 조건에 해당하는 data 조회
sri_err = df_target[ target_column ].apply( cond_all )

sri_fixed_col = df_target.loc[ sri_err ][[target_column]].apply( fix_func ,axis=1  )
df_target.loc[sri_err].head(5)



['Wii', 'PS3', 'PS', 'PS4', 'PS2', 'XB', 'X360', 'GC', '2600', 'SAT', 'NES', 'DC', 'N64', 'XOne', 'SNES', 'GEN', 'SCD', 'NG', 'TG16', '3DO', 'PCFX']


,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Type
1,DS,2008,Action,40000,0,0,0,portable
2,Wii,2009,Action,170000,0,0,10000,console
3,PSP,2010,Adventure,0,0,20000,0,portable
4,DS,2010,Misc,40000,0,0,0,portable
5,PS3,2010,Platform,120000,90000,0,40000,console


In [111]:
# 데이터 클리닝 결과 재확인
df_target = eda_info (df_target, need_category = True , category_count = 50 ) 


------------------------------------------------
# 데이터셋 정보 확인
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Platform     16241 non-null  category
 1   Year         16241 non-null  int64   
 2   Genre        16241 non-null  category
 3   NA_Sales     16241 non-null  int64   
 4   EU_Sales     16241 non-null  int64   
 5   JP_Sales     16241 non-null  int64   
 6   Other_Sales  16241 non-null  int64   
 7   Type         16241 non-null  object  
dtypes: category(2), int64(5), object(1)
memory usage: 1.4+ MB
None

------------------------------------------------
# 데이터의 행열 크기 =  (16241, 8)

------------------------------------------------
결측치가 포함된 Data 갯수 0 / 16241 
각 컬럼별 결측치 갯수
Series([], dtype: float64)
Empty DataFrame
Columns: []
Index: []

------------------------------------------------
# 컬럼별 중복값 확인 
Platform 컬럼에 중복값 16239 개
Year 컬럼에 

### 클리닝 된 데이터를 파일로 저장 

In [114]:
df_target.to_csv('vgames2_after_cleaning_01.csv', index=False)

----
----
# 4. 분석




## 목표 ( 최종 )

**요청사항 : 다음 분기에 어떤 게임을 설계해야 할까?**

    최종 목표 요청사항 상세 정의 
    - 1 단계 
        * 어떤 장르를 개발해야 하는가?
        * 어떤 플렛폼용으로 개발해야 하는가?
        * 어느지역 ( 언어지원 ,  주요 마켓팅 역량 집중 )을 주요 타겟으로 잡아야 하는가?

    - 2 단계 
        * 어떤 장르를 개발해야 하는가?
            -> 어떤 장르의 게임을 개발했을 때 히트 게임이 될 확률이 높은가?
            -> p( Hit 게임 | 장르 ) 
        * 어떤 플렛폼용으로 개발해야 하는가?
            -> 어떤 플렛폼 의 게임을 개발했을 때 히트 게임이 될 확률이 높은가?
            -> p( Hit 게임 | 플렛폼 ) 

        * 어느지역 ( 언어지원 ,  주요 마켓팅 역량 집중 )을 주요 타겟으로 잡아야 하는가?
            -> 어떤 지역을 지원하는 게임을 개발했을 때 히트 게임이 될 확률이 높은가?
            -> p( Hit 게임 | 지역 )

    - 3 단계 
        목표 -> 각 장르별로 매출이 높을 확률 ? 베이지안 
            A : hit 게임
            B : 장르(각)

            P(A|B) = P(B|A) * P(A) / P(B)

        목표 -> 각 플렛폼 별로 매출이 높을 확률  ? 베이지안
            A : hit 게임
            B : 플렛폼(각)

            P(A|B) = P(B|A) * P(A) / P(B)

        목표 -> 각 지역별로 언어지원,집중 마켓팅을 했을때 매출이 높을 확률  ? 베이지안
            A : hit 게임
            B : 지역(각)

            P(A|B) = P(B|A) * P(A) / P(B)








----
----
# 시각화


### 한글 폰트 설치
: 설치 후 메뉴>런타임>런타임 다시 시작 할 것

In [ ]:
# 한글 폰트 설치 
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 폰트 셋팅
plt.style.use('ggplot')
font = {'size': 12,
        'family': 'NanumBarunGothic'}
mpl.rc('font', **font)

----

In [ ]:
# 생존여부, 성별 별 평균요금 df 조회
df_temp = df_titanic.groupby( ['survived' ,'sex'] ).mean()
df_temp

In [ ]:
df_temp

In [ ]:
df_red['color'] = np.repeat('red', len(df_red))
df_white['color'] = np.repeat('white', len(df_white))
df_temp = df_red.append(df_white)

df_temp.groupby('color')['quality'].mean().plot(kind='bar', 
                                                title='Average Wine Quality by Color', 
                                                fontsize=13, 
                                                color=['red', 'blue'], 
                                                alpha=0.4);

In [ ]:
counts = df_temp.groupby(['quality', 'color']).count()['pH']
counts.plot(kind='bar', color=['red','blue'], title='Counts by Wine Quality and Color', fontsize=15, alpha=0.4);

In [ ]:
df_temp = df_titanic

# 막대그래프 표시 
df_temp.plot(kind = 'bar')

In [ ]:
df_temp = df_titanic

temp = df_temp.groupby(['sex', 'survived'])['fare'].mean().reset_index()
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(data=temp, x='survived', y='fare', hue='sex', ax=ax)
ax.set_title("생존 여부와 성별에 따른 평균 요금")
ax.set_xticklabels(['Not Survived', 'Survived'])
plt.show();

In [ ]:
df_temp = df_titanic

fig, ax = plt.subplots(figsize=(8, 8))
fig = plt.pie(df_temp['survived'].value_counts(), labels=['No', 'Yes'], autopct='%.2f%%',
              textprops={'fontsize':14})
plt.title('Survived', fontsize=16)
plt.show()

In [ ]:
df_temp = df_titanic

fig, ax = plt.subplots(figsize=(8, 6))
sns.countplot(x=df_temp['pclass'].values, ax=ax)
plt.title('클래스별 수', fontsize=16)
plt.show()

In [ ]:
sns.countplot(x=df_temp['pclass'].values)

In [ ]:
df_temp['age'].plot(kind='hist', bins=8)

In [ ]:
df_temp = df_titanic

bins = np.arange(0, df_temp.age.max()+df_temp.age.max()/8, df_temp.age.max()/8)
fig, ax = plt.subplots(figsize=(8, 6))
fig = plt.hist(x=df_temp['age'], bins=bins)
plt.title('나이 분포', fontsize=16)
plt.xlabel('나이')
plt.ylabel('수')
plt.show()

In [ ]:
df_temp = df_red

# quality 에 따른 휘발성산의 차이를 scatter plot 으로 확인
plt.scatter(df_temp['quality'], df_temp['volatile acidity'])

In [ ]:
df_temp = df_red

# quality 에 따른 평균 휘발성산 확인
group_volatile_acidity = df_temp['volatile acidity'].groupby(df_temp['quality'])
print(group_volatile_acidity.mean())

# plot으로 시각화
plt.plot(group_volatile_acidity.mean());

----
# # Data Wrangling (데이터 정제) , Data Preprocessing (데이터 전처리)  
  : 실제 작업의 70~80%가 소요 된다고 함 

  * 데이터 품질
    * 결측값 처리
    * 잘못된 값 처리
  * 구조적 품질
    * 정규화



In [ ]:
# DataFrame 복사
df_patients_clean = df_patients.copy()

### Missing value (결측값)

특정 컬럼의 결측값 갯수 확인

In [ ]:
# 환자 데이터 중 키, 몸무게, 체질량지수 컬럼의 결측값 확인
df_patients_error_tmp = df_patients[df_patients['키'].isnull() | df_patients['몸무게'].isnull() | df_patients['체질량지수'].isnull()]
df_patients_error_tmp.head(3)

(TODO) 특정 컬럼의 결측값을 수정


### Outliers (잘못된 값,이상치)  수정


In [ ]:
# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()


* URL 주소 
  : https:// 로 시작 하지 않는 데이터의 앞에 https://를 붙인다. 

In [ ]:
# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Poster_Link'
cond_err = lambda x: True if not x.startswith('https://') else False
fix_func = lambda x: 'https://' + x

# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_err, fix_func ) 



다른 열의 값을 가지고 와서 합치기

In [ ]:
# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Series_Title'
cond_err = lambda x: True 

# case 01 
fix_func = lambda x: df_temp_clean.loc[x.name,"Runtime"] + df_temp_clean.loc[x.name,"Genre"]

# case 02 : 조건 체크 columm 과 변경 column이 다른 경우
def fix_func2 ( x ) :
  df_temp_clean.loc[x.name,"Certificate"] = df_temp_clean.loc[x.name,"Runtime"] + df_temp_clean.loc[x.name,"Genre"]
  return df_temp_clean.loc[x.name,target_column]

df_result = fix_column_err_value( df_temp_clean , target_column , cond_err, fix_func2 ) 




특정 컬럼을 필요 없는 문자 삭제 후 int로 타입 변경 



In [ ]:
# case 01

# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()

df_temp_clean['Runtime'] = df_temp_clean['Runtime'].str.extract('([0-9]+)').astype('int16')
# movie['Runtime'] = movie['Runtime'].str.split().str[0].astype('int16')

In [ ]:
# case 02

# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()


# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Runtime'
cond_sel = lambda x: True
fix_func = lambda x: re.search( r'([0-9]+)' , x).group(0)

# 처리할 값에 대한 예외 처리가 필요한 경우 조건 처리를 포함한 function을 사용하여야 한다. 
def fix_func2( input ):
    if not bool( input ) : 
      return np.nan
    if not isinstance( input ,str ) :
      print( type(input)  ,  input)
      return np.nan
    return re.search( r'([0-9]+)' , input ).group(0)

# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_sel, fix_func ) 
df_result.head()

# Lecture 에서 따온 특수문자 체크 코드

In [ ]:
reg_con_01 = r'[_,!,.,@#$%()^\d+]'

# 특수 문자가 포함되었는지 여부 체크
def has_errors(inputString):
  return bool(re.search( reg_con_01 , inputString))


# 이름에 섞여 있는 오류 데이터를 확인
error_condition = df_patients_clean.이름.apply(has_errors)
error_names = df_patients_clean[error_condition]
print(error_names)

print("-------------")
# 코드가 정확하게 오류들을 제거하는지 확인 
error_names.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')
print(error_names)

print("-------------")
# 반영
df_patients_clean.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')

# 반영결과 확인
df_patients_clean[error_condition] 

### 이메일 형식 체크

In [ ]:
df_discussion_114.head(20)

In [ ]:
regx_이메일 = r'[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*@[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*\.[a-zA-Z]{2,3}'
regx_핸드폰 = r'\d{3}-\d{3,4}-\d{4}'

# 작업 DataFrame 지정
df_temp_clean = df_discussion_114.copy()

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'email'

#cond_sel = lambda x: False if re.search( regx_이메일 , x).group(0) else True
def cond_sel (x) :
  if isinstance ( x , str ) :
   if re.match( regx_이메일 , x) :
    return False
   else :
    return True
  else :
    # str이 아닌 경우= Nan 인 경우 , 수정 대상으로 체크
    return True

def fix_func(x) :
  # 수정 대상인 경우 (TODO) 알맞은 코딩을 해주어야 함
  return "(TODO) Fix"


# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_sel, fix_func ) 

# 수정 결과 확인
display(df_result.head(11))
display(df_result.tail(5))

### (TODO) 전화번호 형식 체크

### 컬럼의 분리

In [ ]:
# 환자나이 컬럼 분리
# 나이 컬럼에서 숫자만 뽑아내어 int로 변환 후 환자나이 컬럼을 생성 
df_patients_clean['환자나이'] = df_patients_clean.나이.str.extract('(\d+)').astype('int') 

# 성별 컬럼 분리
# 나이 컬럼에서 숫자를 모두 제거 후 성별 컬럼을 생성
df_patients_clean['성별'] = df_patients_clean.나이.str.replace(r'[^a-zA-Z]', '', regex=True) #숫자만 삭제 한다.

# 기존 나이 컬럼 제거
patients_clean = df_patients_clean.drop('나이', axis='columns' )


In [ ]:
# 성별을 category 데이터 타입으로 변화 , 영어를 한글로 변환
patients_clean.성별 = patients_clean.성별.replace({'male':'남', 'female':'여'})
patients_clean.성별 = patients_clean.성별.astype('category')

## 구조적 문제
df1.melt(id_vars='index', value_vars=['A', 'B'])

pd.melt(df1, id_vars = 'index', var_name='variable', value_name='value')

# 결과 데이터를 파일로 저장
